<a href="https://colab.research.google.com/github/hackerinheels/dresser/blob/main/us2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This is a notebook that builds a model to identify a traditional dresser against a contemporary or industrial style dresser.

In [1]:
import numpy as np
import os
import PIL
import PIL.Image
import tensorflow as tf
import tensorflow_datasets as tfds

In [ ]:
print(tf.__version__)

2.6.0


In [ ]:
import pathlib
trainingdata_url = "https://storage.googleapis.com/furndata/us.zip"
testdata_url = "https://storage.googleapis.com/furndata/test.zip"
data_dir = tf.keras.utils.get_file(origin=trainingdata_url, 
                                   fname='us', 
                                   untar=True)
data_dir = pathlib.Path(data_dir)
test_data_dir = tf.keras.utils.get_file(origin=testdata_url, 
                                   fname='test', 
                                   untar=True)
test_data_dir = pathlib.Path(test_data_dir)
!unzip /root/.keras/datasets/us.tar.gz -d ./
!unzip /root/.keras/datasets/test.tar.gz -d ./

In [12]:
training_data_dir = "/content/us/"
training_data_dir = pathlib.Path(training_data_dir)
test_data_dir = "/content/test/"
test_data_dir = pathlib.Path(test_data_dir)
# First run unzip data.tar.gz
print(len(list(training_data_dir.glob('*/*'))))
print(len(list(test_data_dir.glob('*/*'))))

122
9


In [10]:
batch_size = 32
img_height = 256
img_width = 256

In [13]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
  training_data_dir,
  validation_split=0.2,
  subset="training",
  labels='inferred',
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 119 files belonging to 2 classes.
Using 96 files for training.


In [14]:
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
  training_data_dir,
  validation_split=0.2,
  subset="validation",
  labels='inferred',
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 119 files belonging to 2 classes.
Using 23 files for validation.


In [15]:
test_ds = tf.keras.preprocessing.image_dataset_from_directory(
  test_data_dir,
  labels='inferred',
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 9 files belonging to 2 classes.


In [16]:
class_names = train_ds.class_names
print(class_names)

['aeshaan', 'me']


In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 10))
for images, labels in train_ds.take(1):
  for i in range(9):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(images[i].numpy().astype("uint8"))
    plt.title(class_names[labels[i]])
    plt.axis("off")

In [28]:
num_classes = 2
IMG_SIZE = 256
model = tf.keras.Sequential([
   
  tf.keras.layers.experimental.preprocessing.Resizing(IMG_SIZE, IMG_SIZE),
  tf.keras.layers.experimental.preprocessing.Rescaling(1./255),
  tf.keras.layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical"),
  tf.keras.layers.experimental.preprocessing.RandomRotation(0.2),
  tf.keras.layers.Conv2D(32, 3, activation='relu'),
  tf.keras.layers.BatchNormalization(),
  tf.keras.layers.MaxPool2D(),
  tf.keras.layers.Conv2D(64, 3, activation='relu'),
  tf.keras.layers.BatchNormalization(),
  tf.keras.layers.MaxPool2D(),
  tf.keras.layers.Dropout(0.1),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(1)
])

In [29]:
model.compile(
  optimizer='adam',
  loss=tf.losses.BinaryCrossentropy(from_logits=True),
  metrics=['accuracy'])

In [30]:
model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=10
)

Epoch 1/10
3/3 [==============================] - 20s 5s/step - loss: 24.5916 - accuracy: 0.5833 - val_loss: 3.0822 - val_accuracy: 0.7391
Epoch 2/10
3/3 [==============================] - 18s 5s/step - loss: 24.1884 - accuracy: 0.6667 - val_loss: 3.6055 - val_accuracy: 0.7391
Epoch 3/10
3/3 [==============================] - 18s 5s/step - loss: 9.4289 - accuracy: 0.8021 - val_loss: 2.1684 - val_accuracy: 0.7391
Epoch 4/10
3/3 [==============================] - 18s 5s/step - loss: 10.7698 - accuracy: 0.7812 - val_loss: 1.0535 - val_accuracy: 0.7391
Epoch 5/10
3/3 [==============================] - 18s 5s/step - loss: 10.2042 - accuracy: 0.7604 - val_loss: 1.2333 - val_accuracy: 0.7826
Epoch 6/10
3/3 [==============================] - 18s 5s/step - loss: 5.1343 - accuracy: 0.8333 - val_loss: 1.4871 - val_accuracy: 0.7391
Epoch 7/10
3/3 [==============================] - 18s 5s/step - loss: 4.2159 - accuracy: 0.8438 - val_loss: 1.9568 - val_accuracy: 0.8261
Epoch 8/10
3/3 [==============

In [31]:
# Evaluate the model on the test data using `evaluate`
print("Evaluate on test data")
results = model.evaluate(test_ds)
print("test loss, test acc:", results)


Evaluate on test data
1/1 [==============================] - 1s 887ms/step - loss: 0.6343 - accuracy: 0.8889
test loss, test acc: [0.6343067288398743, 0.8888888955116272]


In [32]:
# Generate predictions (probabilities -- the output of the last layer)
# on new data using `predict`
print("Generate predictions for 3 samples")
predictions = model.predict(test_ds)
print("predictions shape:", predictions.shape)

Generate predictions for 3 samples
predictions shape: (9, 1)


In [33]:
checkpoint_path = "/content/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

# Train the model with the new callback
model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=1,
  callbacks=[cp_callback]
)
# This may generate warnings related to saving the state of the optimizer.
# These warnings (and similar warnings throughout this notebook)
# are in place to discourage outdated usage, and can be ignored.

3/3 [==============================] - 18s 5s/step - loss: 2.1922 - accuracy: 0.9062 - val_loss: 20.4346 - val_accuracy: 0.2609

Epoch 00001: saving model to /content/cp.ckpt


In [ ]:
# Loads the weights
model.load_weights(checkpoint_path)

# Re-evaluate the model
#loss, acc = model.evaluate(test_images, test_labels, verbose=2)


NOW Trying hyperparameterization automation using Keras Tuner with a diff model

In [42]:
!pip install -q -U keras-tuner

In [46]:

from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.layers import BatchNormalization, experimental
from tensorflow.keras.layers import Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras_tuner.tuners import RandomSearch
import matplotlib.pyplot as plt

In [35]:
# define cnn model
def build_model_1(hp):
	model = Sequential()
	#model.add(experimental.preprocessing.Rescaling(1./255))
	#model.add(experimental.preprocessing.Resizing(256, 256))
	hp_filters = hp.Int('filters', min_value = 32, max_value = 64, step = 32)
	model.add(Conv2D(filters=hp_filters, kernel_size=(3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', input_shape=(256,256, 3)))
	model.add(BatchNormalization())
	model.add(MaxPooling2D((2, 2)))
	model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
	model.add(BatchNormalization())
	model.add(MaxPooling2D((2, 2)))
	model.add(Dropout(0.1))
	model.add(Flatten())
	model.add(Dense(128, activation='relu', kernel_initializer='he_uniform'))
	model.add(Dense(1, activation='softmax'))
	# compile model
	#opt = SGD(lr=0.001, momentum=0.9)
	hp_learning_rate = hp.Choice('learning_rate', values = [1e-2, 1e-3])
	opt = Adam(learning_rate=hp_learning_rate)#, momentum=0.9)
	model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])
	model.summary()
	return model

In [47]:

tuner = RandomSearch(
    build_model_1,
    objective='val_accuracy',
    max_trials=4,
    executions_per_trial=1)
#,
#    directory='my_dir1') #change the directory name here  when rerunning the cell else it gives "Oracle exit error" 


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 256, 256, 32)      896       
_________________________________________________________________
batch_normalization (BatchNo (None, 256, 256, 32)      128       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 128, 128, 32)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 128, 128, 32)      9248      
_________________________________________________________________
batch_normalization_1 (Batch (None, 128, 128, 32)      128       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 64, 64, 32)        0         
_________________________________________________________________
dropout (Dropout)            (None, 64, 64, 32)        0

In [ ]:
!\rm -rf ./untitled_project/oracle.json

In [ ]:
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

In [48]:
tuner.search(train_ds, validation_data=val_ds,
  epochs=3)

Trial 4 Complete [00h 02m 03s]
val_accuracy: 0.739130437374115

Best val_accuracy So Far: 0.739130437374115
Total elapsed time: 00h 05m 52s
INFO:tensorflow:Oracle triggered exit


In [49]:
tuner.results_summary()


Results summary
Results in ./untitled_project
Showing 10 best trials
Objective(name='val_accuracy', direction='max')
Trial summary
Hyperparameters:
filters: 64
learning_rate: 0.001
Score: 0.739130437374115
Trial summary
Hyperparameters:
filters: 32
learning_rate: 0.001
Score: 0.739130437374115
Trial summary
Hyperparameters:
filters: 32
learning_rate: 0.01
Score: 0.739130437374115
Trial summary
Hyperparameters:
filters: 64
learning_rate: 0.01
Score: 0.739130437374115


A few things remaining
1. Use tuner results
2. Add TF displayboard


In [50]:
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

# Load the TensorBoard notebook extension.
%load_ext tensorboard

In [56]:
for x, y in train_ds.take(1):
  print("Shape: ", x.shape)
  print("Label: ", y[0], "->", class_names[y[0]])

Shape:  (32, 256, 256, 3)
Label:  tf.Tensor(1, shape=(), dtype=int32) -> me


In [59]:
#Visualizing a single image
from datetime import datetime
import io
import itertools
from packaging import version

# Clear out any prior log data.
!rm -rf logs

# Sets up a timestamped log directory.
logdir = "logs/train_data/" + datetime.now().strftime("%Y%m%d-%H%M%S")
# Creates a file writer for the log directory.
file_writer = tf.summary.create_file_writer(logdir)

img = x[0]
# Using the file writer, log the reshaped image.
with file_writer.as_default():
  tf.summary.image("Training data", x, step=0)

In [64]:
# Reshape the image for the Summary API.
img = np.reshape(x[0], (-1, 256, 256, 3))
with file_writer.as_default():
  tf.summary.image("Training data", img, step=0)

In [71]:
!pip install tensorboard --ignore-installed



     |████████████████████████████████| 5.6 MB 5.0 MB/s 
     |████████████████████████████████| 1.1 MB 50.0 MB/s 
     |████████████████████████████████| 3.9 MB 42.4 MB/s 
     |████████████████████████████████| 816 kB 39.9 MB/s 
     |████████████████████████████████| 131 kB 73.6 MB/s 
     |████████████████████████████████| 62 kB 757 kB/s 
     |████████████████████████████████| 781 kB 71.7 MB/s 
     |████████████████████████████████| 4.9 MB 43.8 MB/s 
     |████████████████████████████████| 15.7 MB 184 kB/s 
     |████████████████████████████████| 97 kB 5.2 MB/s 
     |████████████████████████████████| 152 kB 57.1 MB/s 
     |████████████████████████████████| 288 kB 53.3 MB/s 
     |████████████████████████████████| 155 kB 38.0 MB/s 
     |████████████████████████████████| 77 kB 4.9 MB/s 
     |████████████████████████████████| 145 kB 49.9 MB/s 
     |████████████████████████████████| 59 kB 6.0 MB/s 
     |████████████████████████████████| 138 kB 56.5 MB/s 
     |█████████████████

In [69]:
with file_writer.as_default():
  # Don't forget to reshape.
  tf.summary.image("32 training data examples", x, max_outputs=25, step=0)

%tensorboard --logdir logs/train_data -bind_all


ERROR: Failed to launch TensorBoard (exited with 2).
Contents of stderr:
usage: tensorboard [-h] [--helpfull] [--logdir PATH] [--logdir_spec PATH_SPEC]
                   [--host ADDR] [--bind_all] [--port PORT]
                   [--reuse_port BOOL] [--load_fast {false,auto,true}]
                   [--extra_data_server_flags EXTRA_DATA_SERVER_FLAGS]
                   [--grpc_creds_type {local,ssl,ssl_dev}]
                   [--grpc_data_provider PORT] [--purge_orphaned_data BOOL]
                   [--db URI] [--db_import] [--inspect] [--version_tb]
                   [--tag TAG] [--event_file PATH] [--path_prefix PATH]
                   [--window_title TEXT] [--max_reload_threads COUNT]
                   [--reload_interval SECONDS] [--reload_task TYPE]
                   [--reload_multifile BOOL]
                   [--reload_multifile_inactive_secs SECONDS]
                   [--generic_data TYPE]
                   [--samples_per_plugin SAMPLES_PER_PLUGIN]
                   [-